In [2]:
#importation
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import sys
import clean_data
import os

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
#loading data and displaying columns
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
#data_path = os.path.join(parent_dir, "data", "Week1_challenge_data_source.csv")
data_path2 = os.path.join(parent_dir, "data", "user_engage.csv")
data_path3 =  os.path.join(parent_dir, "data", "user_exp.csv")
#db = pd.read_csv(data_path)
user_engagement = pd.read_csv(data_path2)
user_exp = pd.read_csv(data_path3)
user_engagement.drop('Unnamed: 0', inplace = True, axis =1)
user_engagement.head()

,Customer Number,Session Frequency,Session Duration,Total Data Volume,Google(UL/DL),Email(UL/DL),Gaming(UL/DL),Youtube(UL/DL),Netflix(UL/DL),Social(UL/DL)
0,3.360100e+10,1,116720.0,8.786906e+08,2905912.0,3701304.0,292426453.0,18355943.0,17855187.0,1570185.0
1,3.360100e+10,1,181230.0,1.568596e+08,4414096.0,937385.0,609920783.0,39359124.0,35565545.0,1933278.0
2,3.360100e+10,1,134969.0,5.959665e+08,10229119.0,3363124.0,229980251.0,34425237.0,23751202.0,1726277.0
3,3.360101e+10,1,49878.0,4.223207e+08,11811761.0,2070983.0,810387875.0,36534765.0,15092588.0,657493.0
4,3.360101e+10,2,37104.0,1.457411e+09,7748843.0,2110349.0,531237049.0,34222253.0,17539799.0,912788.0


In [5]:
usr_eng = user_engagement[['Customer Number' ,'Session Frequency','Session Duration','Total Data Volume']]
column = ['Customer Number' ,'Session Frequency','Session Duration','Total Data Volume']
for c in column:
    usr_eng[c]=user_engagement[c]

usr_eng.head()

,Customer Number,Session Frequency,Session Duration,Total Data Volume
0,3.360100e+10,1,116720.0,8.786906e+08
1,3.360100e+10,1,181230.0,1.568596e+08
2,3.360100e+10,1,134969.0,5.959665e+08
3,3.360101e+10,1,49878.0,4.223207e+08
4,3.360101e+10,2,37104.0,1.457411e+09


In [6]:
to_scale = usr_eng[['Session Frequency','Session Duration','Total Data Volume']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(to_scale)
normalized = pd.DataFrame(scaled, columns=['Session Frequency','Session Duration','Total Data Volume'])

In [7]:
from sklearn.cluster import KMeans
KMean_clust= KMeans(n_clusters=3, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)

KMeans(max_iter=1000, n_clusters=3)

In [8]:
normalized['Cluster']= KMean_clust.labels_

metrics = usr_eng[['Session Frequency','Session Duration','Total Data Volume']]
metrics['Cluster']=KMean_clust.labels_
metrics

,Session Frequency,Session Duration,Total Data Volume,Cluster
0,1,116720.0,8.786906e+08,1
1,1,181230.0,1.568596e+08,1
2,1,134969.0,5.959665e+08,1
3,1,49878.0,4.223207e+08,1
4,2,37104.0,1.457411e+09,0
...,...,...,...,...
106851,1,8810.0,7.146416e+08,1
106852,1,140988.0,4.803073e+08,1
106853,1,877385.0,2.321240e+08,1
106854,1,253030.0,5.962878e+08,1


In [9]:
clus_mean = metrics.groupby('Cluster').mean()
clus_mean

,Session Frequency,Session Duration,Total Data Volume
Cluster,,,
0,2.176267,213616.483959,1.125494e+09
1,1.042238,106985.013467,4.955072e+08
2,4.248708,569517.630258,2.280905e+09


In [10]:
centroids = KMean_clust.cluster_centers_
centroids = min_max_scaler.inverse_transform(centroids)
print(centroids)
usr_eng['Engagement Score'] = np.linalg.norm(usr_eng[['Session Frequency', 'Session Duration','Total Data Volume']].sub(np.array(centroids[0])), axis=1)
usr_eng

[[2.17574189e+00 2.13511494e+05 1.12500080e+09]
 [1.04210825e+00 1.06976349e+05 4.95490323e+08]
 [4.24644084e+00 5.69173587e+05 2.27960985e+09]]


,Customer Number,Session Frequency,Session Duration,Total Data Volume,Engagement Score
0,3.360100e+10,1,116720.0,8.786906e+08,2.463102e+08
1,3.360100e+10,1,181230.0,1.568596e+08,9.681412e+08
2,3.360100e+10,1,134969.0,5.959665e+08,5.290343e+08
3,3.360101e+10,1,49878.0,4.223207e+08,7.026801e+08
4,3.360101e+10,2,37104.0,1.457411e+09,3.324102e+08
...,...,...,...,...,...
106851,3.379000e+10,1,8810.0,7.146416e+08,4.103592e+08
106852,3.379000e+10,1,140988.0,4.803073e+08,6.446935e+08
106853,3.197020e+12,1,877385.0,2.321240e+08,8.928771e+08
106854,3.370000e+14,1,253030.0,5.962878e+08,5.287130e+08


In [11]:
user_exp.head()

,Customer Number,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL),No. of Handset
0,3.360100e+10,76.0,46.000000,2.156957e+07,1
1,3.360100e+10,99.0,31.000000,2.156957e+07,1
2,3.360100e+10,97.0,127.458589,2.156957e+07,1
3,3.360101e+10,248.0,84.000000,7.607247e+05,1
4,3.360101e+10,56844.0,119.000000,3.094040e+07,1


In [12]:
to_scale = user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
scaled = min_max_scaler.fit_transform(to_scale)
normalized = pd.DataFrame(scaled, columns=['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)'])
KMean_clust= KMeans(n_clusters=3, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)
metric=user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']]
metric['Cluster']=KMean_clust.labels_
metric

,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL),Cluster
0,76.0,46.000000,2.156957e+07,0
1,99.0,31.000000,2.156957e+07,0
2,97.0,127.458589,2.156957e+07,0
3,248.0,84.000000,7.607247e+05,0
4,56844.0,119.000000,3.094040e+07,1
...,...,...,...,...
106851,10365.0,52.000000,2.180450e+05,0
106852,116.0,40.000000,2.156957e+07,0
106853,1.0,127.458589,2.156957e+07,0
106854,33.0,127.458589,2.156957e+07,0


In [13]:
clus_mean = metric.groupby('Cluster').mean()
clus_mean

,Avg Bearer TP(UL/DL),Avg RTT(UL/DL),TCP Retrans(UL/DL)
Cluster,,,
0,4769.858154,167.254401,2.142795e+07
1,66996.622940,182.966603,3.928258e+07
2,208614.541420,396.135441,2.391076e+08


In [14]:
usr_eng.columns.tolist()

['Customer Number',
 'Session Frequency',
 'Session Duration',
 'Total Data Volume',
 'Engagement Score']

In [15]:
centroids = KMean_clust.cluster_centers_
centroids = min_max_scaler.inverse_transform(centroids)

satis=user_exp
satis['Experience Score'] = np.linalg.norm( user_exp[['Avg Bearer TP(UL/DL)','Avg RTT(UL/DL)','TCP Retrans(UL/DL)']].sub(np.array(centroids[0])), axis=1)

# retreive engagement scores
satis['Engagement Score']=usr_eng['Engagement Score'].values
col = satis.loc[: , "Engagement Score":"Experience Score"]
satis['Satisfaction'] = user_exp[["Engagement Score","Experience Score"]].mean(axis=1)
satis=satis[["Engagement Score","Experience Score",'Satisfaction']]
satis.nlargest(10,'Satisfaction')


,Engagement Score,Experience Score,Satisfaction
66860,1.080487e+09,8.332778e+09,4.706632e+09
34123,5.278308e+08,7.920430e+09,4.224130e+09
42366,1.047159e+09,7.353468e+09,4.200314e+09
98213,2.137940e+09,5.858558e+09,3.998249e+09
6437,7.721232e+09,1.129878e+08,3.917110e+09
92923,7.389779e+09,2.287761e+08,3.809277e+09
13180,7.374643e+09,2.407523e+08,3.807698e+09
100468,2.043156e+09,5.074255e+09,3.558705e+09
13526,6.846172e+09,2.154193e+08,3.530796e+09
76363,6.766111e+09,1.459225e+08,3.456017e+09


In [16]:
from sklearn.linear_model import LinearRegression

x = satis[['Engagement Score','Experience Score']]
y = satis['Satisfaction']
x, y = np.array(x), np.array(y)
model = LinearRegression().fit(x, y)
r_sq = model.score(x, y)
print('coefficient determination:', r_sq)
print('intercept:', model.intercept_)
print('slope:', model.coef_)
y_pred = model.predict(x)
print('predicted response:', y_pred, sep='\n')

coefficient determination: 1.0
intercept: -5.960464477539063e-08
slope: [0.5 0.5]
predicted response:
[1.23224599e+08 4.84140054e+08 2.64586637e+08 ... 4.46508015e+08
 2.64425965e+08 4.92993164e+08]


In [17]:
x = satis[['Engagement Score','Experience Score']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized = pd.DataFrame(x_scaled,columns=['Engagement Score','Experience Score'])
KMean_clust= KMeans(n_clusters=2, init='k-means++',max_iter=1000)
KMean_clust.fit(normalized)
satis['Cluster']=KMean_clust.labels_
satis

,Engagement Score,Experience Score,Satisfaction,Cluster
0,2.463102e+08,1.389532e+05,1.232246e+08,1
1,9.681412e+08,1.389524e+05,4.841401e+08,0
2,5.290343e+08,1.389524e+05,2.645866e+08,1
3,7.026801e+08,2.066997e+07,3.616750e+08,0
4,3.324102e+08,9.509849e+06,1.709600e+08,1
...,...,...,...,...
106851,4.103592e+08,2.121265e+07,2.157859e+08,1
106852,6.446935e+08,1.389518e+05,3.224162e+08,0
106853,8.928771e+08,1.389557e+05,4.465080e+08,0
106854,5.287130e+08,1.389546e+05,2.644260e+08,1


In [18]:
satis.groupby('Cluster')[['Engagement Score','Experience Score']].mean()

,Engagement Score,Experience Score
Cluster,,
0,8.502540e+08,1.970726e+07
1,3.462806e+08,1.912712e+07


In [29]:
satis.head()

,Engagement Score,Experience Score,Satisfaction,Cluster
0,2.463102e+08,1.389532e+05,1.232246e+08,1
1,9.681412e+08,1.389524e+05,4.841401e+08,0
2,5.290343e+08,1.389524e+05,2.645866e+08,1
3,7.026801e+08,2.066997e+07,3.616750e+08,0
4,3.324102e+08,9.509849e+06,1.709600e+08,1


In [30]:
import sqlite3
con = sqlite3.connect ('tellco.db')
cursor = con.cursor ()
try:
    cur.execute ('create table user_satisfaction_level (user_id, eng_score number, exp_score number, Satisfaction)')
except :
    print('Table exists.... continuing')


satis['user_id']=satis.index
satis[['user_id','Engagement Score','Experience Score','Satisfaction']].to_sql('user_satisfaction_level', con, if_exists='replace', index=False)

satis

Table exists.... continuing


,Engagement Score,Experience Score,Satisfaction,Cluster,user_id
0,2.463102e+08,1.389532e+05,1.232246e+08,1,0
1,9.681412e+08,1.389524e+05,4.841401e+08,0,1
2,5.290343e+08,1.389524e+05,2.645866e+08,1,2
3,7.026801e+08,2.066997e+07,3.616750e+08,0,3
4,3.324102e+08,9.509849e+06,1.709600e+08,1,4
...,...,...,...,...,...
106851,4.103592e+08,2.121265e+07,2.157859e+08,1,106851
106852,6.446935e+08,1.389518e+05,3.224162e+08,0,106852
106853,8.928771e+08,1.389557e+05,4.465080e+08,0,106853
106854,5.287130e+08,1.389546e+05,2.644260e+08,1,106854


In [31]:
cursor.execute("SELECT * FROM user_satisfaction_level")

rows = cursor.fetchall()
rows=rows[:-10]
for row in rows:
        print(row)
   
con.close ()

(0, 246310243.93846053, 138953.17132056845, 123224598.55489054)
(1, 968141156.4587815, 138952.41156871556, 484140054.4351751)
(2, 529034321.75095004, 138952.4175901972, 264586637.08427012)
(3, 702680119.9732976, 20669974.66321373, 361675047.31825566)
(4, 332410191.8884557, 9509848.721402626, 170960020.30492917)
(5, 509783579.5270958, 901298.2575416104, 255342438.8923187)
(6, 470277740.62962484, 251950.47452860145, 235264845.55207673)
(7, 792340452.1167012, 20670762.46935783, 406505607.29302955)
(8, 134868905.13900208, 21708447.51694722, 78288676.32797465)
(9, 392537047.93257445, 619507.7669139463, 196578277.8497442)
(10, 1010024763.5239804, 138953.1534783883, 505081858.3387294)
(11, 494908395.89443403, 6836069.201534627, 250872232.54798433)
(12, 642581727.417675, 138953.40160239217, 321360340.4096387)
(13, 766088899.4621419, 138955.19439043588, 383113927.32826614)
(14, 894676403.9796735, 138952.38402510065, 447407678.1818493)
(15, 583220860.3796234, 20638122.53377618, 301929491.4566998

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

